In [2]:
!pip install speechbrain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.2/807.2 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 31.7 MB/s eta 0:00:00


In [3]:
!pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [4]:
!pip install pydub

In [5]:
pip install --upgrade speechbrain


In [6]:
import requests
import time
from pydub import AudioSegment
import numpy as np
from speechbrain.inference import EncoderClassifier


In [7]:
api_key = "dhjsg"

In [27]:
def upload_audio(file_path):
    headers = {
        'authorization': api_key,
    }

    with open(file_path, 'rb') as f:
        response = requests.post('https://api.assemblyai.com/v2/upload', headers=headers, files={'file': f})

    return response.json()['upload_url']

In [28]:
# Step 2: Request transcription with speaker diarization enabled
def request_transcription(audio_url):
    endpoint = "https://api.assemblyai.com/v2/transcript"

    json_data = {
        "audio_url": audio_url,
        "speaker_diarization": True,
        "punctuate": False,
        "format_text": False
    }

    headers = {
        "authorization": api_key,
        "content-type": "application/json"
    }

    response = requests.post(endpoint, json=json_data, headers=headers)

    # Check for errors in the response
    if response.status_code != 200:
        print("Error in transcription request:", response.json())
        raise Exception("Failed to request transcription.")

    return response.json()['id']

In [29]:
def check_status(transcript_id):
    endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    headers = {
        "authorization": api_key
    }

    response = requests.get(endpoint, headers=headers)
    return response.json()

In [30]:
def get_speaker_timestamps(transcript_id):
    while True:
        result = check_status(transcript_id)
        if result['status'] == 'completed':
            # Parse speaker timestamps from the "utterances" key
            speaker_segments = result.get('utterances', [])
            speaker_timestamps = [
                {
                    "speaker": segment["speaker"],
                    "start_time": segment["start"],
                    "end_time": segment["end"]
                }
                for segment in speaker_segments
            ]
            return speaker_timestamps
        elif result['status'] == 'failed':
            raise Exception("Transcription failed.")
        else:
            print("Processing... Please wait.")
            time.sleep(5)

In [31]:
def split_audio_segments(timestamps, audio):
    speaker_audio_segments = {}

    for segment in timestamps:
        start_time = segment["start_time"] * 1000  # convert to ms
        end_time = segment["end_time"] * 1000      # convert to ms
        speaker = segment["speaker"]

        speaker_segment = audio[start_time:end_time]

        if speaker not in speaker_audio_segments:
            speaker_audio_segments[speaker] = [speaker_segment]
        else:
            speaker_audio_segments[speaker].append(speaker_segment)

    return speaker_audio_segments


In [32]:
# Step 6: Perform emotion analysis using speechbrain
def analyze_emotion(audio_segment, classifier):
    # Convert pydub AudioSegment to a numpy array
    samples = np.array(audio_segment.get_array_of_samples())
    samples = samples.astype(np.float32) / (2 ** 15)  # Normalize to -1 to 1

    # Save the audio segment temporarily as a WAV file
    temp_audio_file = "temp_audio.wav"
    audio_segment.export(temp_audio_file, format="wav")

    # Run the emotion classifier on the temporary file
    emotion_prediction = classifier.classify_file(temp_audio_file)
    return emotion_prediction

In [33]:
# Main function to handle the entire process
def main(file_path):
    # Step 1: Upload the audio file
    audio_url = upload_audio(file_path)

    # Step 2: Request transcription with speaker diarization
    transcript_id = request_transcription(audio_url)

    # Step 3: Retrieve speaker timestamps
    speaker_timestamps = get_speaker_timestamps(transcript_id)

    # Step 4: Load the audio using pydub
    audio = AudioSegment.from_mp3(file_path)

    # Step 5: Split the audio into segments for each speaker
    speaker_segments = split_audio_segments(speaker_timestamps, audio)

    # Step 6: Load the emotion recognition model from speechbrain
    classifier = EncoderClassifier.from_hparams(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", savedir="tmp")

    # Step 7: Perform emotion analysis for each speaker's segments
    for speaker, segments in speaker_segments.items():
        for segment in segments:
            emotion = analyze_emotion(segment, classifier)
            print(f"Emotion for {speaker}: {emotion}")


In [34]:
# Example usage:
if __name__ == "__main__":
    file_path = "/content/ElevenLabs_2024-09-17T17_23_57_Laura_pre_s50_sb75_se0_b_m2.mp3"  # Replace with your audio file path
    main(file_path)

Error in transcription request: {'error': 'Invalid endpoint schema, please refer to documentation for examples.'}


Exception: Failed to request transcription.

In [22]:
######

In [26]:
import assemblyai as aai
import numpy as np
from pydub import AudioSegment
from speechbrain.inference import EncoderClassifier

# Set your API key
aai.settings.api_key = "f69817768510455f96f4088d628065fb"

# Use a local file path or a publicly accessible URL
audio_file = "/content/ElevenLabs_2024-09-17T17_23_57_Laura_pre_s50_sb75_se0_b_m2.mp3"  # Replace with your audio file path

# Configure the transcription settings to enable speaker labels
config = aai.TranscriptionConfig(
    speaker_labels=True,
)

# Configure the transcription settings to enable speaker labels
config = aai.TranscriptionConfig(
    speaker_labels=True,
)

# Transcribe the audio file
transcript = aai.Transcriber().transcribe(audio_file, config)

# Load the audio for segmentation
audio = AudioSegment.from_mp3(audio_file)

# Prepare the emotion classifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", savedir="tmp")

# Analyze emotion for each utterance
for utterance in transcript.utterances:
    start_time = int(utterance.start)  # Start time in milliseconds
    end_time = int(utterance.end)      # End time in milliseconds
    speaker = utterance.speaker

    # Extract the segment
    segment = audio[start_time:end_time]

    # Save the segment to a temporary file for emotion analysis
    temp_audio_file = "temp_audio.wav"
    segment.export(temp_audio_file, format="wav")


    # Analyze emotion
    # Directly classify the audio file
    emotion_prediction = classifier.classify_file(temp_audio_file)

    # Print results
    print(f"Speaker {speaker}: {utterance.text}")
    print(f"Emotion for Speaker {speaker}: {emotion_prediction}")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It 

AttributeError: 'ModuleDict' object has no attribute 'compute_features'

In [36]:
import requests
import time
from pydub import AudioSegment
import numpy as np
from speechbrain.inference import EncoderClassifier

# Your AssemblyAI API key
api_key = "f69817768510455f96f4088d628065fb"  # Replace with your actual API key

# Step 1: Upload the audio file to AssemblyAI
def upload_audio(file_path):
    headers = {
        'authorization': api_key,
    }

    with open(file_path, 'rb') as f:
        response = requests.post('https://api.assemblyai.com/v2/upload', headers=headers, files={'file': f})

    if response.status_code != 200:
        print("Error uploading audio:", response.json())
        raise Exception("Failed to upload audio.")

    return response.json()['upload_url']


def request_transcription(audio_url):
    endpoint = "https://api.assemblyai.com/v2/transcript"

    json_data = {
        "audio_url": audio_url,
        "speaker_diarization": True,
        "punctuate": True,
        "format_text": True
    }

    headers = {
        "authorization": api_key,
        "content-type": "application/json"
    }

    print("Audio URL:", audio_url)  # Log the audio URL
    print("Request JSON:", json_data)  # Log the JSON payload

    response = requests.post(endpoint, json=json_data, headers=headers)

    if response.status_code != 200:
        print("Error in transcription request:", response.json())
        raise Exception(f"Failed to request transcription: {response.json()}")  # Include response in exception

    return response.json().get('id', None)



# Step 3: Check transcription status and retrieve timestamps
def check_status(transcript_id):
    endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"

    headers = {
        "authorization": api_key
    }

    response = requests.get(endpoint, headers=headers)
    return response.json()


# Step 4: Extract speaker timestamps
def get_speaker_timestamps(transcript_id):
    while True:
        result = check_status(transcript_id)
        if result['status'] == 'completed':
            speaker_segments = result.get('utterances', [])
            speaker_timestamps = [
                {
                    "speaker": segment["speaker"],
                    "start_time": segment["start"],
                    "end_time": segment["end"]
                }
                for segment in speaker_segments
            ]
            return speaker_timestamps
        elif result['status'] == 'failed':
            print("Transcription failed:", result)
            raise Exception("Transcription failed.")
        else:
            print("Processing... Please wait.")
            time.sleep(5)


# Step 5: Split the audio file into segments based on speaker timestamps
def split_audio_segments(timestamps, audio):
    speaker_audio_segments = {}

    for segment in timestamps:
        start_time = segment["start_time"] * 1000  # convert to ms
        end_time = segment["end_time"] * 1000      # convert to ms
        speaker = segment["speaker"]

        speaker_segment = audio[start_time:end_time]

        if speaker not in speaker_audio_segments:
            speaker_audio_segments[speaker] = [speaker_segment]
        else:
            speaker_audio_segments[speaker].append(speaker_segment)

    return speaker_audio_segments


# Step 6: Perform emotion analysis using speechbrain
def analyze_emotion(audio_segment, classifier):
    samples = np.array(audio_segment.get_array_of_samples())
    samples = samples.astype(np.float32) / (2 ** 15)

    temp_audio_file = "temp_audio.wav"
    audio_segment.export(temp_audio_file, format="wav")

    emotion_prediction = classifier.classify_file(temp_audio_file)
    return emotion_prediction


# Main function to handle the entire process
def main(file_path):
    audio_url = upload_audio(file_path)

    transcript_id = request_transcription(audio_url)

    speaker_timestamps = get_speaker_timestamps(transcript_id)

    audio = AudioSegment.from_mp3(file_path)

    speaker_segments = split_audio_segments(speaker_timestamps, audio)

    classifier = EncoderClassifier.from_hparams(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", savedir="tmp")

    for speaker, segments in speaker_segments.items():
        for segment in segments:
            emotion = analyze_emotion(segment, classifier)
            print(f"Emotion for {speaker}: {emotion}")

# Example usage:
if __name__ == "__main__":
    file_path = "/content/ElevenLabs_2024-09-17T17_23_57_Laura_pre_s50_sb75_se0_b_m2.mp3"  # Replace with your audio file path
    main(file_path)


Audio URL: https://cdn.assemblyai.com/upload/c729fa13-d1d2-45f2-bf79-d5650a8f420b
Request JSON: {'audio_url': 'https://cdn.assemblyai.com/upload/c729fa13-d1d2-45f2-bf79-d5650a8f420b', 'speaker_diarization': True, 'punctuate': True, 'format_text': True}
Error in transcription request: {'error': 'Invalid endpoint schema, please refer to documentation for examples.'}


Exception: Failed to request transcription: {'error': 'Invalid endpoint schema, please refer to documentation for examples.'}

In [38]:
import requests
import time

# Step 1: Set up your API key and base URL
api_key = "f69817768510455f96f4088d628065fb"  # Replace with your actual API key
base_url = "https://api.assemblyai.com/v2"

# Step 2: Set up the headers for the API requests
headers = {
    "authorization": api_key
}

# Step 3: Upload your audio file
def upload_audio(file_path):
    with open(file_path, "rb") as f:
        response = requests.post(base_url + "/upload", headers=headers, data=f)

    if response.status_code != 200:
        print("Error uploading audio:", response.json())
        raise Exception("Failed to upload audio.")

    return response.json()["upload_url"]

# Step 4: Request transcription using the uploaded audio URL
def request_transcription(upload_url):
    data = {
        "audio_url": upload_url
    }

    response = requests.post(base_url + "/transcript", json=data, headers=headers)

    if response.status_code != 200:
        print("Error in transcription request:", response.json())
        raise Exception("Failed to request transcription.")

    return response.json()["id"]

# Step 5: Poll the API for the transcription result
def poll_transcription(transcript_id):
    polling_endpoint = f"{base_url}/transcript/{transcript_id}"

    while True:
        transcription_result = requests.get(polling_endpoint, headers=headers).json()

        if transcription_result['status'] == 'completed':
            return transcription_result
        elif transcription_result['status'] == 'error':
            raise RuntimeError(f"Transcription failed: {transcription_result['error']}")
        else:
            time.sleep(3)

# Main function to handle the workflow
def main(file_path):
    try:
        # Upload audio file
        upload_url = upload_audio(file_path)

        # Request transcription
        transcript_id = request_transcription(upload_url)

        # Poll for transcription result
        result = poll_transcription(transcript_id)

        # Print the transcription text
        print("Transcription Text:", result['text'])
        print("Confidence Score:", result['confidence'])

    except Exception as e:
        print("An error occurred:", e)

# Example usage
if __name__ == "__main__":
    audio_file_path = "/content/ElevenLabs_2024-09-17T17_23_57_Laura_pre_s50_sb75_se0_b_m2.mp3"  # Replace with your audio file path
    main(audio_file_path)


Transcription Text: And finally, theres the negative prompt box. This is where you can specify what you dont want in your video. So, for example, if I dont want the puppy to interact with any other animals, I could mention that here to avoid distractions.
Confidence Score: 0.940107


In [39]:
import assemblyai as aai

# Set your API key
aai.settings.api_key = "f69817768510455f96f4088d628065fb"

# Use a publicly-accessible audio URL or a local file
audio_file = (
    "https://github.com/AssemblyAI-Community/audio-examples/raw/main/20230607_me_canadian_wildfires.mp3"
)

# Configure the transcription with speaker labels and sentiment analysis
config = aai.TranscriptionConfig(
    speaker_labels=True,
    sentiment_analysis=True
)

# Transcribe the audio file
transcript = aai.Transcriber().transcribe(audio_file, config)

# Extract and display results
for utterance in transcript.utterances:
    # Print speaker and their spoken text
    print(f"Speaker {utterance.speaker}: {utterance.text}")

    # Extract sentiment analysis for each utterance
    for sentiment_result in transcript.sentiment_analysis:
        if sentiment_result.start >= utterance.start and sentiment_result.end <= utterance.end:
            print(f"Speaker {utterance.speaker} - Sentiment: {sentiment_result.sentiment} "
                  f"(Confidence: {sentiment_result.confidence}) "
                  f"Timestamp: {sentiment_result.start} - {sentiment_result.end}")



Speaker A: Smoke from hundreds of wildfires in Canada is triggering air quality alerts throughout the US. Skylines from Maine to Maryland to Minnesota are gray and smoggy. And in some places, the air quality warnings include the warning to stay inside. We wanted to better understand what's happening here and why. So he called Peter DiCarlo, an associate professor in the department of Environmental Health and Engineering at Johns Hopkins University. Good morning. Professor.
Speaker A - Sentiment: NEGATIVE (Confidence: 0.818103) Timestamp: 240 - 6262
Speaker A - Sentiment: NEGATIVE (Confidence: 0.5900679) Timestamp: 6446 - 10998
Speaker A - Sentiment: NEUTRAL (Confidence: 0.537145) Timestamp: 11094 - 15734
Speaker A - Sentiment: NEUTRAL (Confidence: 0.79514164) Timestamp: 15902 - 18390
Speaker A - Sentiment: NEUTRAL (Confidence: 0.9258168) Timestamp: 18430 - 25334
Speaker A - Sentiment: POSITIVE (Confidence: 0.7193592) Timestamp: 25502 - 26006
Speaker A - Sentiment: NEUTRAL (Confidence: 

In [42]:
import assemblyai as aai

# Set your API key
aai.settings.api_key = "f69817768510455f96f4088d628065fb"

# Use a publicly-accessible audio URL or a local file
audio_file = (
    "https://github.com/AssemblyAI-Community/audio-examples/raw/main/20230607_me_canadian_wildfires.mp3"
)

# Configure the transcription with speaker labels and sentiment analysis
config = aai.TranscriptionConfig(
    speaker_labels=True,
    sentiment_analysis=True
)

# Transcribe the audio file
transcript = aai.Transcriber().transcribe(audio_file, config)

# Initialize sentiment counters for each speaker dynamically
speaker_sentiments = {}

# Extract and display results
for utterance in transcript.utterances:
    speaker_id = utterance.speaker  # e.g., 'A' or 'B'

    # Initialize counters for the speaker if not already present
    if speaker_id not in speaker_sentiments:
        speaker_sentiments[speaker_id] = {'positive': 0, 'negative': 0, 'neutral': 0}

    # Print speaker and their spoken text
    print(f"Speaker {speaker_id}: {utterance.text}")

    # Extract sentiment analysis for each utterance
    for sentiment_result in transcript.sentiment_analysis:
        if sentiment_result.start >= utterance.start and sentiment_result.end <= utterance.end:
            sentiment = sentiment_result.sentiment

            # Update sentiment counters for the corresponding speaker
            if sentiment == "POSITIVE":
                speaker_sentiments[speaker_id]['positive'] += 1
            elif sentiment == "NEGATIVE":
                speaker_sentiments[speaker_id]['negative'] += 1
            elif sentiment == "NEUTRAL":
                speaker_sentiments[speaker_id]['neutral'] += 1

            print(f"Speaker {speaker_id} - Sentiment: {sentiment} "
                  f"(Confidence: {sentiment_result.confidence}) "
                  f"Timestamp: {sentiment_result.start} - {sentiment_result.end}")

# Print overall sentiment for each speaker
print("\nOverall Sentiment Analysis:")
for speaker, sentiments in speaker_sentiments.items():
    total = sum(sentiments.values())
    overall_sentiment = max(sentiments, key=sentiments.get)
    print(f"{speaker}: {overall_sentiment.capitalize()} (Counts - Positive: {sentiments['positive']}, "
          f"Negative: {sentiments['negative']}, Neutral: {sentiments['neutral']})")


Speaker A: Smoke from hundreds of wildfires in Canada is triggering air quality alerts throughout the US. Skylines from Maine to Maryland to Minnesota are gray and smoggy. And in some places, the air quality warnings include the warning to stay inside. We wanted to better understand what's happening here and why. So he called Peter DiCarlo, an associate professor in the department of Environmental Health and Engineering at Johns Hopkins University. Good morning. Professor.
Speaker A - Sentiment: NEGATIVE (Confidence: 0.818103) Timestamp: 240 - 6262
Speaker A - Sentiment: NEGATIVE (Confidence: 0.5900679) Timestamp: 6446 - 10998
Speaker A - Sentiment: NEUTRAL (Confidence: 0.537145) Timestamp: 11094 - 15734
Speaker A - Sentiment: NEUTRAL (Confidence: 0.79514164) Timestamp: 15902 - 18390
Speaker A - Sentiment: NEUTRAL (Confidence: 0.9258168) Timestamp: 18430 - 25334
Speaker A - Sentiment: POSITIVE (Confidence: 0.7193592) Timestamp: 25502 - 26006
Speaker A - Sentiment: NEUTRAL (Confidence: 

In [55]:
import assemblyai as aai
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import soundfile as sf
import librosa

# Set your API key
aai.settings.api_key = "f69817768510455f96f4088d628065fb"

# Use a publicly-accessible audio URL or a local file
audio_file = (
    "/content/20230607_me_canadian_wildfires.mp3"
)

# Configure the transcription with speaker labels and sentiment analysis
config = aai.TranscriptionConfig(
    speaker_labels=True,
    sentiment_analysis=True
)


# Transcribe the audio file
transcript = aai.Transcriber().transcribe(audio_file, config)

# Load the pretrained model and processor for emotion recognition
emotion_model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-large-superb-er")
emotion_processor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-large-superb-er")

# # Function to classify emotions from audio
# def classify_emotion(audio_path):
#     audio_input, _ = librosa.load(audio_path, sr=16000)
#     inputs = emotion_processor(audio_input, return_tensors="pt", sampling_rate=16000)

#     with torch.no_grad():
#         logits = emotion_model(**inputs).logits

#     predicted_ids = torch.argmax(logits, dim=-1)
#     return predicted_ids.item()  # Adjust based on your model's output format

# import numpy as np

# # Function to classify emotions from audio
# def classify_emotion(audio_path):
#     audio_input, _ = librosa.load(audio_path, sr=16000)

#     # Check if audio length is less than the minimum required length (adjust as needed)
#     min_length = emotion_processor.config.input_features[0]['sequence_length']  # Example to get the required length
#     if len(audio_input) < min_length:
#         # Pad the audio with zeros (silence) if it's too short
#         padding_length = min_length - len(audio_input)
#         audio_input = np.pad(audio_input, (0, padding_length), 'constant')

#     inputs = emotion_processor(audio_input, return_tensors="pt", sampling_rate=16000)

#     with torch.no_grad():
#         logits = emotion_model(**inputs).logits

#     predicted_ids = torch.argmax(logits, dim=-1)
#     return predicted_ids.item()  # Adjust based on your model's output format

def classify_emotion(audio_path):
    audio_input, _ = librosa.load(audio_path, sr=16000)

    # Define a common input length for Wav2Vec2 models, usually 2 seconds at 16kHz
    target_length = 32000  # 2 seconds of audio at 16000 Hz

    # Check if audio length is less than the target length
    if len(audio_input) < target_length:
        # Pad the audio with zeros (silence) if it's too short
        padding_length = target_length - len(audio_input)
        audio_input = np.pad(audio_input, (0, padding_length), 'constant')
    else:
        # Trim the audio if it's too long
        audio_input = audio_input[:target_length]

    # Prepare the inputs for the model
    inputs = emotion_processor(audio_input, return_tensors="pt", sampling_rate=16000)

    with torch.no_grad():
        logits = emotion_model(**inputs).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    return predicted_ids.item()  # Adjust based on your model's output format

# Initialize sentiment counters for each speaker
speaker_sentiments = {}
speaker_emotions = {}

# Process each utterance
for utterance in transcript.utterances:
    speaker_id = utterance.speaker  # e.g., 'A' or 'B'

    # Initialize counters for the speaker if not already present
    if speaker_id not in speaker_sentiments:
        speaker_sentiments[speaker_id] = {'positive': 0, 'negative': 0, 'neutral': 0}
        speaker_emotions[speaker_id] = []

    # Print speaker and their spoken text
    print(f"Speaker {speaker_id}: {utterance.text}")

    # Extract sentiment analysis for each utterance
    for sentiment_result in transcript.sentiment_analysis:
        if sentiment_result.start >= utterance.start and sentiment_result.end <= utterance.end:
            sentiment = sentiment_result.sentiment

            # Update sentiment counters for the corresponding speaker
            if sentiment == "POSITIVE":
                speaker_sentiments[speaker_id]['positive'] += 1
            elif sentiment == "NEGATIVE":
                speaker_sentiments[speaker_id]['negative'] += 1
            elif sentiment == "NEUTRAL":
                speaker_sentiments[speaker_id]['neutral'] += 1

            print(f"Speaker {speaker_id} - Sentiment: {sentiment} "
                  f"(Confidence: {sentiment_result.confidence}) "
                  f"Timestamp: {sentiment_result.start} - {sentiment_result.end}")

    # Classify emotion for the utterance
    # Segment audio for the utterance
    start_time = int(utterance.start * 1000)  # convert to milliseconds
    end_time = int(utterance.end * 1000)  # convert to milliseconds
    segment_audio_path = f"{speaker_id}_segment.wav"  # Update to a valid path for each segment

    # Extract the segment using librosa and save it
    audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time / 1000, duration=(end_time - start_time) / 1000)
    sf.write(segment_audio_path, audio_input, 16000)

    # Classify emotion for this segment
    emotion_label = classify_emotion(segment_audio_path)

    # Store emotion label
    speaker_emotions[speaker_id].append(emotion_label)

# Print overall sentiment for each speaker
print("\nOverall Sentiment Analysis:")
for speaker, sentiments in speaker_sentiments.items():
    total = sum(sentiments.values())
    overall_sentiment = max(sentiments, key=sentiments.get)
    print(f"{speaker}: {overall_sentiment.capitalize()} (Counts - Positive: {sentiments['positive']}, "
          f"Negative: {sentiments['negative']}, Neutral: {sentiments['neutral']})")

# Print overall emotion classification for each speaker
print("\nOverall Tonal Classification:")
for speaker, emotions in speaker_emotions.items():
    emotion_counts = {emotion: emotions.count(emotion) for emotion in set(emotions)}
    most_common_emotion = max(emotion_counts, key=emotion_counts.get)
    print(f"{speaker}: {most_common_emotion} (Counts - {emotion_counts})")

Some weights of the model checkpoint at superb/wav2vec2-large-superb-er were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at superb/wav2vec2-large-superb-er and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_

Speaker A: Smoke from hundreds of wildfires in Canada is triggering air quality alerts throughout the US. Skylines from Maine to Maryland to Minnesota are gray and smoggy. And in some places, the air quality warnings include the warning to stay inside. We wanted to better understand what's happening here and why. So he called Peter DiCarlo, an associate professor in the department of Environmental Health and Engineering at Johns Hopkins University. Good morning. Professor.
Speaker A - Sentiment: NEGATIVE (Confidence: 0.818103) Timestamp: 240 - 6262
Speaker A - Sentiment: NEGATIVE (Confidence: 0.5900679) Timestamp: 6446 - 10998
Speaker A - Sentiment: NEUTRAL (Confidence: 0.537145) Timestamp: 11094 - 15734
Speaker A - Sentiment: NEUTRAL (Confidence: 0.79514164) Timestamp: 15902 - 18390
Speaker A - Sentiment: NEUTRAL (Confidence: 0.9258168) Timestamp: 18430 - 25334
Speaker A - Sentiment: POSITIVE (Confidence: 0.7193592) Timestamp: 25502 - 26006
Speaker A - Sentiment: NEUTRAL (Confidence: 

<ipython-input-55-f5e538abdde1>:125: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time / 1000, duration=(end_time - start_time) / 1000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Speaker A: So what is it about the conditions right now that have caused this round of wildfires to affect so many people so far away?
Speaker A - Sentiment: NEGATIVE (Confidence: 0.7462095) Timestamp: 28938 - 37350


<ipython-input-55-f5e538abdde1>:125: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time / 1000, duration=(end_time - start_time) / 1000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


...

Speaker A: That's Peter DiCarlo, associate professor in the department of Environmental Health and engineering at Johns Hopkins University. Professor de Carlo, thanks so much for joining us and sharing this expertise with us.
Speaker A - Sentiment: NEUTRAL (Confidence: 0.9091453) Timestamp: 267976 - 274576
Speaker A - Sentiment: POSITIVE (Confidence: 0.97781634) Timestamp: 274688 - 278460


<ipython-input-55-f5e538abdde1>:125: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time / 1000, duration=(end_time - start_time) / 1000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Speaker B: Thank you for having me.
Speaker B - Sentiment: POSITIVE (Confidence: 0.97090584) Timestamp: 279440 - 280280


<ipython-input-55-f5e538abdde1>:125: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time / 1000, duration=(end_time - start_time) / 1000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



Overall Sentiment Analysis:
A: Neutral (Counts - Positive: 2, Negative: 9, Neutral: 15)
B: Neutral (Counts - Positive: 3, Negative: 7, Neutral: 21)

Overall Tonal Classification:
A: 2 (Counts - {0: 1, 2: 9})
B: 2 (Counts - {2: 10})


In [13]:
import assemblyai as aai
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import soundfile as sf
import librosa
import numpy as np

# Set your API key for AssemblyAI
aai.settings.api_key = "f69817768510455f96f4088d628065fb"

# Use a publicly-accessible audio URL or a local file
audio_file = "/content/20230607_me_canadian_wildfires.mp3"

# Configure transcription with speaker labels and sentiment analysis
config = aai.TranscriptionConfig(
    speaker_labels=True,
    sentiment_analysis=True
)

# Transcribe the audio file
transcript = aai.Transcriber().transcribe(audio_file, config)

# Load the pretrained model and processor for emotion recognition
emotion_model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-large-superb-er")
emotion_processor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-large-superb-er")

# Define emotion labels based on your model's output
emotion_labels = {
    0: "neutral",
    1: "happy",
    2: "angry",
    3: "sad"
}

# Function to classify emotions from audio
def classify_emotion(audio_path):
    audio_input, _ = librosa.load(audio_path, sr=16000)

    # Define a common input length for Wav2Vec2 models, usually 2 seconds at 16kHz
    target_length = 32000  # 2 seconds of audio at 16000 Hz

    # Check if audio length is less than the target length
    if len(audio_input) < target_length:
        # Pad the audio with zeros (silence) if it's too short
        padding_length = target_length - len(audio_input)
        audio_input = np.pad(audio_input, (0, padding_length), 'constant')
    else:
        # Trim the audio if it's too long
        audio_input = audio_input[:target_length]

    # Prepare the inputs for the model
    inputs = emotion_processor(audio_input, return_tensors="pt", sampling_rate=16000)

    # Get emotion prediction
    with torch.no_grad():
        logits = emotion_model(**inputs).logits

    predicted_id = torch.argmax(logits, dim=-1).item()

    # Map the predicted ID to the corresponding emotion label
    emotion_label = emotion_labels.get(predicted_id, "unknown")

    return emotion_label

# Initialize sentiment counters and emotion lists for each speaker
speaker_sentiments = {}
speaker_emotions = {}

# Process each utterance in the transcription
for utterance in transcript.utterances:
    speaker_id = utterance.speaker  # e.g., 'A' or 'B'

    # Initialize counters and lists for the speaker if not already present
    if speaker_id not in speaker_sentiments:
        speaker_sentiments[speaker_id] = {'positive': 0, 'negative': 0, 'neutral': 0}
        speaker_emotions[speaker_id] = []

    # Print speaker and their spoken text
    print(f"Speaker {speaker_id}: {utterance.text}")

    # Extract sentiment analysis for each utterance
    for sentiment_result in transcript.sentiment_analysis:
        if sentiment_result.start >= utterance.start and sentiment_result.end <= utterance.end:
            sentiment = sentiment_result.sentiment

            # Update sentiment counters for the corresponding speaker
            if sentiment == "POSITIVE":
                speaker_sentiments[speaker_id]['positive'] += 1
            elif sentiment == "NEGATIVE":
                speaker_sentiments[speaker_id]['negative'] += 1
            elif sentiment == "NEUTRAL":
                speaker_sentiments[speaker_id]['neutral'] += 1

            print(f"Speaker {speaker_id} - Sentiment: {sentiment} "
                  f"(Confidence: {sentiment_result.confidence}) "
                  f"Timestamp: {sentiment_result.start} - {sentiment_result.end}")

    # Segment audio for the utterance
    start_time = utterance.start  # in seconds
    end_time = utterance.end  # in seconds
    segment_audio_path = f"{speaker_id}_segment_{start_time}.wav"  # Unique file name

    # Extract the audio segment for this utterance
    audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
    sf.write(segment_audio_path, audio_input, 16000)

    # Classify emotion for this segment
    emotion_label = classify_emotion(segment_audio_path)

    # Store the emotion label for the speaker
    speaker_emotions[speaker_id].append(emotion_label)

# Print overall sentiment analysis for each speaker
print("\nOverall Sentiment Analysis:")
for speaker, sentiments in speaker_sentiments.items():
    total = sum(sentiments.values())
    overall_sentiment = max(sentiments, key=sentiments.get)
    print(f"{speaker}: {overall_sentiment.capitalize()} (Counts - Positive: {sentiments['positive']}, "
          f"Negative: {sentiments['negative']}, Neutral: {sentiments['neutral']})")

# Print overall emotion classification for each speaker
print("\nOverall Tonal Classification:")
for speaker, emotions in speaker_emotions.items():
    emotion_counts = {emotion: emotions.count(emotion) for emotion in set(emotions)}
    most_common_emotion = max(emotion_counts, key=emotion_counts.get)
    print(f"{speaker}: {most_common_emotion} (Counts - {emotion_counts})")


Some weights of the model checkpoint at superb/wav2vec2-large-superb-er were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at superb/wav2vec2-large-superb-er and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_

Speaker A: Smoke from hundreds of wildfires in Canada is triggering air quality alerts throughout the US. Skylines from Maine to Maryland to Minnesota are gray and smoggy. And in some places, the air quality warnings include the warning to stay inside. We wanted to better understand what's happening here and why. So he called Peter DiCarlo, an associate professor in the department of Environmental Health and Engineering at Johns Hopkins University. Good morning. Professor.
Speaker A - Sentiment: NEGATIVE (Confidence: 0.818103) Timestamp: 240 - 6262
Speaker A - Sentiment: NEGATIVE (Confidence: 0.5900679) Timestamp: 6446 - 10998
Speaker A - Sentiment: NEUTRAL (Confidence: 0.537145) Timestamp: 11094 - 15734
Speaker A - Sentiment: NEUTRAL (Confidence: 0.79514164) Timestamp: 15902 - 18390
Speaker A - Sentiment: NEUTRAL (Confidence: 0.9258168) Timestamp: 18430 - 25334
Speaker A - Sentiment: POSITIVE (Confidence: 0.7193592) Timestamp: 25502 - 26006
Speaker A - Sentiment: NEUTRAL (Confidence: 

<ipython-input-13-e1811ccfe674>:104: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Speaker A: So what is it about the conditions right now that have caused this round of wildfires to affect so many people so far away?
Speaker A - Sentiment: NEGATIVE (Confidence: 0.7462095) Timestamp: 28938 - 37350


<ipython-input-13-e1811ccfe674>:104: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


...

Speaker A: That's Peter DiCarlo, associate professor in the department of Environmental Health and engineering at Johns Hopkins University. Professor de Carlo, thanks so much for joining us and sharing this expertise with us.
Speaker A - Sentiment: NEUTRAL (Confidence: 0.9091453) Timestamp: 267976 - 274576
Speaker A - Sentiment: POSITIVE (Confidence: 0.97781634) Timestamp: 274688 - 278460


<ipython-input-13-e1811ccfe674>:104: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Speaker B: Thank you for having me.
Speaker B - Sentiment: POSITIVE (Confidence: 0.97090584) Timestamp: 279440 - 280280


<ipython-input-13-e1811ccfe674>:104: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)



Overall Sentiment Analysis:
A: Neutral (Counts - Positive: 2, Negative: 9, Neutral: 15)
B: Neutral (Counts - Positive: 3, Negative: 7, Neutral: 21)

Overall Tonal Classification:
A: angry (Counts - {'angry': 9, 'neutral': 1})
B: angry (Counts - {'angry': 10})


In [10]:
# Get the model's configuration
model_config = emotion_model.config

# Retrieve the label-to-emotion mapping
id2label = model_config.id2label

# Print the mapping
print("Emotion ID to Label Mapping:")
print(id2label)



Emotion ID to Label Mapping:
{0: 'neu', 1: 'hap', 2: 'ang', 3: 'sad'}


In [11]:
# without transcription
import assemblyai as aai
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import soundfile as sf
import librosa
import numpy as np

# Set your AssemblyAI API key for speaker diarization and sentiment analysis
aai.settings.api_key = "f69817768510455f96f4088d628065fb"

# Load your audio file
audio_file = "/content/20230607_me_canadian_wildfires.mp3"

# Configure diarization and sentiment analysis (without transcription)
config = aai.TranscriptionConfig(
    speaker_labels=True,  # For speaker diarization
    sentiment_analysis=True  # For sentiment analysis
)

# Perform speaker diarization and sentiment analysis using AssemblyAI
transcript = aai.Transcriber().transcribe(audio_file, config)

# Load the pretrained model and processor for emotion recognition
emotion_model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-large-superb-er")
emotion_processor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-large-superb-er")

# Define the function for classifying emotions from audio segments
def classify_emotion(audio_path):
    audio_input, _ = librosa.load(audio_path, sr=16000)

    # Define a common input length for Wav2Vec2 models (e.g., 2 seconds)
    target_length = 32000  # 2 seconds of audio at 16000 Hz

    # Pad or trim the audio to fit the target length
    if len(audio_input) < target_length:
        padding_length = target_length - len(audio_input)
        audio_input = np.pad(audio_input, (0, padding_length), 'constant')
    else:
        audio_input = audio_input[:target_length]

    # Prepare inputs for the emotion model
    inputs = emotion_processor(audio_input, return_tensors="pt", sampling_rate=16000)

    with torch.no_grad():
        logits = emotion_model(**inputs).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    return predicted_ids.item()

# Initialize sentiment counters for each speaker
speaker_sentiments = {}
speaker_emotions = {}

# Process speaker diarization and sentiment analysis
for utterance in transcript.utterances:
    speaker_id = utterance.speaker  # Get speaker ID

    # Initialize sentiment and emotion tracking for this speaker
    if speaker_id not in speaker_sentiments:
        speaker_sentiments[speaker_id] = {'positive': 0, 'negative': 0, 'neutral': 0}
        speaker_emotions[speaker_id] = []

    # Handle sentiment analysis for this utterance
    for sentiment_result in transcript.sentiment_analysis:
        if sentiment_result.start >= utterance.start and sentiment_result.end <= utterance.end:
            sentiment = sentiment_result.sentiment

            # Update sentiment counts
            if sentiment == "POSITIVE":
                speaker_sentiments[speaker_id]['positive'] += 1
            elif sentiment == "NEGATIVE":
                speaker_sentiments[speaker_id]['negative'] += 1
            elif sentiment == "NEUTRAL":
                speaker_sentiments[speaker_id]['neutral'] += 1

    # Perform emotion classification on the audio segment
    start_time = int(utterance.start * 1000)  # Convert to milliseconds
    end_time = int(utterance.end * 1000)

    segment_audio_path = f"{speaker_id}_segment.wav"  # Path for audio segment

    # Extract the audio segment using librosa
    audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time / 1000, duration=(end_time - start_time) / 1000)
    sf.write(segment_audio_path, audio_input, 16000)

    # Classify the emotion for this audio segment
    emotion_label = classify_emotion(segment_audio_path)

    # Store the classified emotion for the speaker
    speaker_emotions[speaker_id].append(emotion_label)

# Print overall sentiment and emotion analysis
print("\nOverall Sentiment Analysis:")
for speaker, sentiments in speaker_sentiments.items():
    overall_sentiment = max(sentiments, key=sentiments.get)
    print(f"Speaker {speaker}: {overall_sentiment.capitalize()}")

# print("\nOverall Tonal Classification:")
# for speaker, emotions in speaker_emotions.items():
#     emotion_counts = {emotion: emotions.count(emotion) for emotion in set(emotions)}
#     most_common_emotion = max(emotion_counts, key=emotion_counts.get)
#     print(f"Speaker {speaker}: {most_common_emotion} (Counts: {emotion_counts})")

# Print overall tonal classification with emotion labels
print("\nOverall Tonal Classification:")
for speaker, emotions in speaker_emotions.items():
    emotion_counts = {emotion: emotions.count(emotion) for emotion in set(emotions)}
    most_common_emotion = max(emotion_counts, key=emotion_counts.get)

    # Convert emotion ID to label
    emotion_label = id2label[most_common_emotion]

    # Print results with emotion labels instead of numbers
    print(f"Speaker {speaker}: {emotion_label} (Counts: {emotion_counts})")


Some weights of the model checkpoint at superb/wav2vec2-large-superb-er were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at superb/wav2vec2-large-superb-er and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_


Overall Sentiment Analysis:
Speaker A: Neutral
Speaker B: Neutral

Overall Tonal Classification:
Speaker A: ang (Counts: {0: 1, 2: 9})
Speaker B: ang (Counts: {2: 10})


In [14]:
#

import assemblyai as aai
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor
import torch
import soundfile as sf
import librosa
import numpy as np

# Set your API key for AssemblyAI
aai.settings.api_key = "f69817768510455f96f4088d628065fb"

# Use a publicly-accessible audio URL or a local file
audio_file = "/content/20230607_me_canadian_wildfires.mp3"

# Configure transcription with speaker labels and sentiment analysis
config = aai.TranscriptionConfig(
    speaker_labels=True,
    sentiment_analysis=True
)

# Transcribe the audio file
transcript = aai.Transcriber().transcribe(audio_file, config)

# Load the pretrained model and processor for emotion recognition
emotion_model = Wav2Vec2ForSequenceClassification.from_pretrained("superb/wav2vec2-large-superb-er")
emotion_processor = Wav2Vec2FeatureExtractor.from_pretrained("superb/wav2vec2-large-superb-er")

# Define emotion labels based on your model's output
emotion_labels = {
    0: "neutral",
    1: "happy",
    2: "angry",
    3: "sad"
}

# Function to classify emotions from audio
def classify_emotion(audio_path):
    audio_input, _ = librosa.load(audio_path, sr=16000)

    # Define a common input length for Wav2Vec2 models, usually 2 seconds at 16kHz
    target_length = 32000  # 2 seconds of audio at 16000 Hz

    # Check if audio length is less than the target length
    if len(audio_input) < target_length:
        # Pad the audio with zeros (silence) if it's too short
        padding_length = target_length - len(audio_input)
        audio_input = np.pad(audio_input, (0, padding_length), 'constant')
    else:
        # Trim the audio if it's too long
        audio_input = audio_input[:target_length]

    # Prepare the inputs for the model
    inputs = emotion_processor(audio_input, return_tensors="pt", sampling_rate=16000)

    # Get emotion prediction
    with torch.no_grad():
        logits = emotion_model(**inputs).logits

    predicted_id = torch.argmax(logits, dim=-1).item()

    # Map the predicted ID to the corresponding emotion label
    emotion_label = emotion_labels.get(predicted_id, "unknown")

    return emotion_label

# Initialize sentiment counters and emotion lists for each speaker
speaker_sentiments = {}
speaker_emotions = {}

# Process each utterance in the transcription
for utterance in transcript.utterances:
    speaker_id = utterance.speaker  # e.g., 'A' or 'B'

    # Initialize counters and lists for the speaker if not already present
    if speaker_id not in speaker_sentiments:
        speaker_sentiments[speaker_id] = {'positive': 0, 'negative': 0, 'neutral': 0}
        speaker_emotions[speaker_id] = []

    # Print speaker and their spoken text
    print(f"Speaker {speaker_id}: {utterance.text}")

    # Extract sentiment analysis for each utterance
    for sentiment_result in transcript.sentiment_analysis:
        if sentiment_result.start >= utterance.start and sentiment_result.end <= utterance.end:
            sentiment = sentiment_result.sentiment

            # Update sentiment counters for the corresponding speaker
            if sentiment == "POSITIVE":
                speaker_sentiments[speaker_id]['positive'] += 1
            elif sentiment == "NEGATIVE":
                speaker_sentiments[speaker_id]['negative'] += 1
            elif sentiment == "NEUTRAL":
                speaker_sentiments[speaker_id]['neutral'] += 1

            print(f"Speaker {speaker_id} - Sentiment: {sentiment} "
                  f"(Confidence: {sentiment_result.confidence}) "
                  f"Timestamp: {sentiment_result.start} - {sentiment_result.end}")

    # Segment audio for the utterance and classify emotion for this segment
    start_time = utterance.start  # in seconds
    end_time = utterance.end  # in seconds
    segment_audio_path = f"{speaker_id}_segment_{start_time}.wav"  # Unique file name

    # Extract the audio segment for this utterance
    audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
    sf.write(segment_audio_path, audio_input, 16000)

    # Classify emotion for this segment
    emotion_label = classify_emotion(segment_audio_path)

    # Print emotion label for each utterance
    print(f"Speaker {speaker_id} - Emotion: {emotion_label} (Segment: {start_time}s - {end_time}s)")

    # Store the emotion label for the speaker
    speaker_emotions[speaker_id].append(emotion_label)

# Print overall sentiment analysis for each speaker
print("\nOverall Sentiment Analysis:")
for speaker, sentiments in speaker_sentiments.items():
    total = sum(sentiments.values())
    overall_sentiment = max(sentiments, key=sentiments.get)
    print(f"{speaker}: {overall_sentiment.capitalize()} (Counts - Positive: {sentiments['positive']}, "
          f"Negative: {sentiments['negative']}, Neutral: {sentiments['neutral']})")

# Print overall emotion classification for each speaker
print("\nOverall Tonal Classification:")
for speaker, emotions in speaker_emotions.items():
    emotion_counts = {emotion: emotions.count(emotion) for emotion in set(emotions)}
    most_common_emotion = max(emotion_counts, key=emotion_counts.get)
    print(f"{speaker}: {most_common_emotion} (Counts - {emotion_counts})")


Some weights of the model checkpoint at superb/wav2vec2-large-superb-er were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at superb/wav2vec2-large-superb-er and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_

Speaker A: Smoke from hundreds of wildfires in Canada is triggering air quality alerts throughout the US. Skylines from Maine to Maryland to Minnesota are gray and smoggy. And in some places, the air quality warnings include the warning to stay inside. We wanted to better understand what's happening here and why. So he called Peter DiCarlo, an associate professor in the department of Environmental Health and Engineering at Johns Hopkins University. Good morning. Professor.
Speaker A - Sentiment: NEGATIVE (Confidence: 0.818103) Timestamp: 240 - 6262
Speaker A - Sentiment: NEGATIVE (Confidence: 0.5900679) Timestamp: 6446 - 10998
Speaker A - Sentiment: NEUTRAL (Confidence: 0.537145) Timestamp: 11094 - 15734
Speaker A - Sentiment: NEUTRAL (Confidence: 0.79514164) Timestamp: 15902 - 18390
Speaker A - Sentiment: NEUTRAL (Confidence: 0.9258168) Timestamp: 18430 - 25334
Speaker A - Sentiment: POSITIVE (Confidence: 0.7193592) Timestamp: 25502 - 26006
Speaker A - Sentiment: NEUTRAL (Confidence: 

<ipython-input-14-8073865f900c>:106: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Speaker B - Emotion: angry (Segment: 27850s - 28786s)
Speaker A: So what is it about the conditions right now that have caused this round of wildfires to affect so many people so far away?
Speaker A - Sentiment: NEGATIVE (Confidence: 0.7462095) Timestamp: 28938 - 37350


<ipython-input-14-8073865f900c>:106: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


...

Speaker B - Emotion: angry (Segment: 241320s - 267792s)
Speaker A: That's Peter DiCarlo, associate professor in the department of Environmental Health and engineering at Johns Hopkins University. Professor de Carlo, thanks so much for joining us and sharing this expertise with us.
Speaker A - Sentiment: NEUTRAL (Confidence: 0.9091453) Timestamp: 267976 - 274576
Speaker A - Sentiment: POSITIVE (Confidence: 0.97781634) Timestamp: 274688 - 278460


<ipython-input-14-8073865f900c>:106: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Speaker A - Emotion: angry (Segment: 267976s - 278460s)
Speaker B: Thank you for having me.
Speaker B - Sentiment: POSITIVE (Confidence: 0.97090584) Timestamp: 279440 - 280280


<ipython-input-14-8073865f900c>:106: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_input, _ = librosa.load(audio_file, sr=16000, offset=start_time, duration=end_time - start_time)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Speaker B - Emotion: angry (Segment: 279440s - 280280s)

Overall Sentiment Analysis:
A: Neutral (Counts - Positive: 2, Negative: 9, Neutral: 15)
B: Neutral (Counts - Positive: 3, Negative: 7, Neutral: 21)

Overall Tonal Classification:
A: angry (Counts - {'angry': 9, 'neutral': 1})
B: angry (Counts - {'angry': 10})
